# Overview

# Imports

In [ ]:
import pandas as pd
import sqlalchemy as sa

In [ ]:
%run sql_magic.ipynb

# Start database

In [ ]:
%run start_db.ipynb

In [ ]:
engine = sa.create_engine(DB_URL, connect_args={'options': '-csearch_path=University'})

In [ ]:
engine.table_names()

In [ ]:
DB_URL

# Previous midterms

> **Disclaimer**
>
> Downloaded from http://oxdia.com, which is in the questionable business of charing money for material which they did not create and do not own.

## Fall 2017 - Midterm Test (L0101)

### Question 1

(a) According to the schema, can a student take the same course more than once?

In [ ]:
#

yes

(b) Consider this constraint:

![](tutorial_7/L0101-q1-b.png)

Define an instance of Offering that violates the constraint.

#

oID | dept | cNum | term | instructor 
- | - | - | - | -
o1 | CSC | 343 | termA | Craig
o2 | CSC | 343 | termA | Horton
o3 | CSC | 108 | termB | Craig
o4 | CSC | 108 | termB | Horton

(c) Write the following constraint using relational algebra: No CSC course may count towards the breadth
requirement (that is, have breadth = True) unless it is a 100-level course.

#

```
\select_{dept = 'CSC' and breadth and cNum > 199} Course = empty
```

### Question 2

#### Write a query in relational algebra to find the sID of every student who has taken each CSC course that a has ever been offered.

Select all CSC courses that have ever been offered:

```
CSCCourses(dept, cnum) := 
    \project_{dept, cnum}
    \select_{dept = 'CSC'}
    Offering
```

Select all courses that all students have taken:

```
StudentTook(sid, dept, cNum) :=
    \project_{sid, dept, cNum}
    (Took \natural_join Offering)
```

Imagine that all students have taken every CSC course:

```
ImaginaryUniverse(sid, dept, cnum) :=
    \project_{sid} StudentTook
    \product
    \CSCCourses
```

Find all rows from `ImaginaryUniverse` which fall short of reality:

```
Missing := ImaginaryUniverse - StudentTook
```

Return all `sID`s that are not in `Missing`:

```
Answer(sid) := 
    (\project_{sid} StudentTook) -
    (\project_{sid} Missing)
```

(Alternative way using division):

```
Answer(sid) = StudentTook \divide CSCCourses
```

### Question 3

#### Suppose we want to find the cNum of the first CSC course that was ever taught, that is, the one with the a minimum value for term. If there was a tie, we want to report the cNums of all the tied courses.

The following query attempts to solve this. It is syntactically correct, but doesn’t always produce the right answer.

```
CSCTerms(cNum) :=
    \project_{cNum}
    \select_{dept = 'csc'}
    Offering

NotFirst(cNum) :=
    \project_{c2.cnum} \select_{c1.term < c2.term}
    (\rename_{t1} CSCTerms \product \rename_{t2} CSCTerms)

Answer(cnum) = CSCTerms - NotFirst
```

### Question 4

### Question 5

### Question 6

In [ ]:
# Create tables
data = {
    'a': ['sina'] * 2 + ['diane'] * 4 + ['michelle'] * 3,
    'b': ['kanyewest', 'RonConwayFacts', 'LilaFontes', 'swcarpentry', 'mfeathers', 'sina', 'sina', 'diane', 'Jeff']
}
follows_df = pd.DataFrame(data, columns=['a', 'b'])
follows_df.to_sql('follows', engine, index=False, if_exists='replace')


data = {
    'id': ['alan', 'sina', 'diane', 'michelle'], 
    'name': ['catman', 'superman', 'superwoman', 'rockstar'],
    'location': ['Ottawa', None, 'Toronto', 'Montreal'],
}
profile_df = pd.DataFrame(data, columns=['id', 'name', 'location'])
profile_df.to_sql('profile', engine, index=False, if_exists='replace')


data = {
    'id': [123, 125, 126, 128, 476, 553],
    'userid': ['alan'] * 4 + ['sina', 'diane'],
    'content': ['hellow twitter', 'bye twitter', 'hellow twitter', 'bye twitter', 'hellow twitter', 'hellow twitter'],
}
profile_df = pd.DataFrame(data, columns=['id', 'userid', 'content'])
profile_df.to_sql('tweets', engine, index=False, if_exists='replace')

In [29]:
%%sql
select * from follows;

,a,b
0,sina,kanyewest
1,sina,RonConwayFacts
2,diane,LilaFontes
3,diane,swcarpentry
4,diane,mfeathers
5,diane,sina
6,michelle,sina
7,michelle,diane
8,michelle,Jeff


In [30]:
%%sql
select * from profile;

,id,name,location
0,alan,catman,Ottawa
1,sina,superman,None
2,diane,superwoman,Toronto
3,michelle,rockstar,Montreal


In [31]:
%%sql
select * from tweets;

,id,userid,content
0,123,alan,hellow twitter
1,125,alan,bye twitter
2,126,alan,hellow twitter
3,128,alan,bye twitter
4,476,sina,hellow twitter
5,553,diane,hellow twitter


In [32]:
%%sql
select count(*)
from profile left join follows
    on a = id;

,count
0,10


In [ ]:
1 + 2 + 4 + 3

In [33]:
%%sql
SELECT P.id, count(T.content) AS number
FROM Profile P JOIN Tweets t
    ON T.userid = P.id AND P.location='Toronto'
GROUP BY(p.id);

,id,number
0,diane,1


profile:

id | name | location
-- | ---- | --------
diane | superwoman | Toronto

tweets:

id | userid | content
--- | ----- | -------
553 | diane | hellow twitter

In [34]:
%%sql
SELECT id, count(b) AS followers
FROM Profile JOIN Follows
ON a = id;

(psycopg2.ProgrammingError) column "profile.id" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT id, count(b) AS followers
               ^
 [SQL: 'SELECT id, count(b) AS followers\nFROM Profile JOIN Follows\nON a = id;']


id | followers
-- | ---------


In [ ]:
%%sql
select location from Follows, Profile
where id = a and b = 'sina';

follows:

a | b
- | -
diane | sina
michelle | sina

profile:

id | name | location
-- | ---- | --------
diane | superwoman | Toronto
michelle | rockstar | Montreal

after select:

a | b | id | name | location
- | - | -- | ---- | --------
diane | sina | diane | superwoman | Toronto
michelle | sina | michelle | rockstar | Montreal

after project:

location |
-------- |
Toronto  |
Montreal |